In [3]:
api_key = "c09f91126e51468d88f57cb83a63ee36"
azure_endpoint = "https://chat-gpt-a1.openai.azure.com/"
api_version = "2023-03-15-preview"

In [2]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="DanielChatGPT16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [5]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key="c09f91126e51468d88f57cb83a63ee36",
    azure_endpoint="https://chat-gpt-a1.openai.azure.com/",
    api_version="2023-03-15-preview",
)

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
documents = SimpleDirectoryReader(
    input_files=["Midsummer Night's Dream.txt"]
).load_data()


In [10]:
from llama_index.core.node_parser import SentenceWindowNodeParser
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
nodes = node_parser.get_nodes_from_documents(documents)

In [11]:
i=10
print(f"Text: \n{nodes[i].text}")
print("------------------")
print(f"Window: \n{nodes[i].metadata['window']}")

Text: 
Stand forth, Demetrius. 
------------------
Window: 
Enter EGEUS, HERMIA, LYSANDER, and DEMETRIUS
EGEUS
     Happy be Theseus, our renowned duke!
 THESEUS
     Thanks, good Egeus: what's the news with thee?
 EGEUS
     Full of vexation come I, with complaint
     Against my child, my daughter Hermia.
      Stand forth, Demetrius.  My noble lord,
     This man hath my consent to marry her.
      Stand forth, Lysander: and my gracious duke,
     This man hath bewitch'd the bosom of my child;
     Thou, thou, Lysander, thou hast given her rhymes,
     And interchanged love-tokens with my child:
     Thou hast by moonlight at her window sung,
     With feigning voice verses of feigning love,
     And stolen the impression of her fantasy
     With bracelets of thy hair, rings, gawds, conceits,
     Knacks, trifles, nosegays, sweetmeats, messengers
     Of strong prevailment in unharden'd youth:
     With cunning hast thou filch'd my daughter's heart,
     Turn'd her obedience, which 

In [12]:
index = VectorStoreIndex(
    nodes
)

INFO:httpx:HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HT

In [13]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

In [14]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    top_n = 2, 
    model = "BAAI/bge-reranker-base"
)

c:\Users\shree\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\shree\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
query_engine = index.as_query_engine(
    similarity_top_k = 6, 
    alpha=0.5,
    node_postprocessors = [postproc, rerank],
)

In [18]:
response = query_engine.query(
    "Why were Oberon and Titania quarrelling?"
)
print(str(response))

INFO:httpx:HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"


Batches: 100%|██████████| 1/1 [00:04<00:00,  4.56s/it]


INFO:httpx:HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/DanielChatGPT16k/chat/completions?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://chat-gpt-a1.openai.azure.com//openai/deployments/DanielChatGPT16k/chat/completions?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
Oberon and Titania were quarrelling because Oberon wanted a changeling boy to be his henchman, but Titania refused to give him the child.


In [19]:
window = response.source_nodes[0].node.metadata["window"]
sentence = response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: And make him with fair AEgle break his faith,
     With Ariadne and Antiopa?
 TITANIA
     These are the forgeries of jealousy:
     And never, since the middle summer's spring,
     Met we on hill, in dale, forest or mead,
     By paved fountain or by rushy brook,
     Or in the beached margent of the sea,
     To dance our ringlets to the whistling wind,
     But with thy brawls thou hast disturb'd our sport.
      Therefore the winds, piping to us in vain,
     As in revenge, have suck'd up from the sea
     Contagious fogs; which falling in the land
     Have every pelting river made so proud
     That they have overborne their continents:
     The ox hath therefore stretch'd his yoke in vain,
     The ploughman lost his sweat, and the green corn
     Hath rotted ere his youth attain'd a beard;
     The fold stands empty in the drowned field,
     And crows are fatted with the murrion flock;
     The nine men's morris is fill'd up with mud,
     And the quaint mazes in the 